<a href="https://colab.research.google.com/github/parwisenlared/MasterThesis/blob/master/Single_Objective_Evolutionary_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing dataset

GA adapted from https://github.com/AlessandroSaviolo/Evolving-CNNs-using-GA

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/b&w" -d "/content/drive/My Drive/Colab Notebooks"

unzip:  cannot find or open /content/drive/My Drive/Colab Notebooks/b&w, /content/drive/My Drive/Colab Notebooks/b&w.zip or /content/drive/My Drive/Colab Notebooks/b&w.ZIP.


In [5]:
import glob, os
#os.chdir("..")
print(os.getcwd())

os.listdir(os.getcwd())
os.chdir("drive/My Drive/Colab Notebooks/")
print(os.getcwd())

/content
/content/drive/My Drive/Colab Notebooks


In [6]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("images/greyscaleZeroPressure/PentagonSpiralDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("images/greyscaleZeroPressure/PentagonSpiralDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

607


(1310, 1310)

In [7]:
def split_dataset(x,y):

    # train and test set (90% training, 10% test)
    X_train = X[:int(0.9*int(len(X)))]
    y_train = y[:int(0.9*int(len(X)))]
    X_test = X[int(0.9*int(len(X))):]
    y_test = y[int(0.9*int(len(y))):]
    
    # validation set (90% training, 10% validation)
    X_train = X_train[:int(0.9*int(len(X_train)))]
    y_train = y_train[:int(0.9*int(len(y_train)))]
    
    X_val = X_train[int(0.9*int(len(X_train))):]
    y_val = y_train[int(0.9*int(len(y_train))):]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

In [8]:
def prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val):

    # reshape training sets to prepare for CNN
    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np.array(y_test)
    
    X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_val = np.array(y_val)
    
    # one hot encoding of pixels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_val = to_categorical(y_val)
    
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    val_norm = X_val.astype('float32')

    # normalize to range 0-1
    X_train = train_norm / 255.0
    X_test = test_norm / 255.0
    X_val = val_norm / 255.0
    # return dataset
    
    return X_train, y_train, X_test, y_test, X_val, y_val

## Evolving CNN with GA algorithm

Code adapted from: 

In [9]:
#Utilities

from keras.datasets import cifar10
from keras.utils import to_categorical
import pickle
import sys
from matplotlib import pyplot as plt


def load_dataset(X, y, batch_size, num_classes, epochs):  					# retrieve CIFAR10 dataset and process data

    x_train, y_train, x_test, y_test, x_val, y_val = split_dataset(X,y)
    
    x_train, y_train, x_test, y_test, x_val, y_val = prepare_dataset(x_train, y_train, x_test, y_test, x_val, y_val)

    dataset = {
        'batch_size': batch_size,
        'num_classes': num_classes,
        'epochs': epochs,
        'x_train': x_train,
        'x_test': x_test,
        'y_train': y_train,
        'y_test': y_test,
        'x_val': x_val,
        'y_val': y_val
    }

    return dataset


def save_network(network):
    object_file = open(network.name + '.obj', 'wb')
    pickle.dump(network, object_file)


def load_network(name):
    object_file = open(name + '.obj', 'rb')
    return pickle.load(object_file)


def order_indexes(self):
    i = 0
    for block in self.block_list:
        block.index = i
        i += 1


def plot_training(history):                                           # plot diagnostic learning curves
    plt.figure(figsize=[8, 6])											# loss curves
    plt.plot(history.history['loss'], 'r', linewidth=3.0)
    plt.plot(history.history['val_loss'], 'b', linewidth=3.0)
    plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
    plt.xlabel('Epochs ', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.title('Loss Curves', fontsize=16)

    filename = sys.argv[0].split('/')[-1]
    plt.savefig('/content/drive/My Drive/Colab Notebooks/plots/' + filename + '_loss_plot.png')

    plt.figure(figsize=[8, 6])											# accuracy curves
    plt.plot(history.history['acc'], 'r', linewidth=3.0)
    plt.plot(history.history['val_acc'], 'b', linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
    plt.xlabel('Epochs ', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.title('Accuracy Curves', fontsize=16)

    filename = sys.argv[0].split('/')[-1]
    plt.savefig('/content/drive/My Drive/Colab Notebooks/' + filename + '_acc_plot.png')
    plt.close()


def plot_statistics(stats):
    plt.figure(figsize=[8, 6])											# fitness curves
    plt.plot([s[0] for s in stats], 'r', linewidth=3.0)
    plt.plot([stats[0][0]] * len(stats), 'b', linewidth=3.0)
    plt.legend(['BestFitness', 'InitialFitness'], fontsize=18)
    plt.xlabel('Generations', fontsize=16)
    plt.ylabel('FitnessValue', fontsize=16)
    plt.title('Fitness Curve', fontsize=16)
    filename = sys.argv[0].split('/')[-1]
    plt.savefig('/content/drive/My Drive/Colab Notebooks/plots/' + filename + '_fitness_plot.png')

    plt.figure(figsize=[8, 6])											# parameters curves
    plt.plot([s[1] for s in stats], 'r', linewidth=3.0)
    plt.plot([stats[0][1]] * len(stats), 'b', linewidth=3.0)
    plt.legend(['BestParamsNum', 'InitialParamsNum'], fontsize=18)
    plt.xlabel('Generations', fontsize=16)
    plt.ylabel('ParamsNum', fontsize=16)
    plt.title('Parameters Curve', fontsize=16)
    filename = sys.argv[0].split('/')[-1]
    plt.savefig('/content/drive/My Drive/Colab Notebooks/' + filename + '_params_plot.png')
    plt.close()


## Topologies

In [10]:
# Topologies

import keras.layers
from random import randint


class Block:
	__slots__ = ('type', 'index', 'layerList1', 'layerList2')

	def __init__(self, type, index, layerList1, layerList2):
		self.type = type											# 0 -> initial layer; 1 -> mid layers; 2 -> final layer
		self.index = index										# block index among all the blocks
		self.layerList1 = layerList1							# Convolutional layers
		self.layerList2 = layerList2							# Pooling and Dropout layers

	def get_layers(self):
		return self.layerList1 + self.layerList2

	def get_size(self):
		return len(self.get_layers())
'''
class Initial:
	def _init_(self, input_shape):
		self.name = "Define input"
		self.input_shape = input_shape

	def build_layer(self, model):
		model.add(keras.Input(input_shape = self.input_shape))
'''
class InitConvolutional:
	# __slots__ = ('name', 'filters', 'padding', 'filter_size')

	def __init__(self, filters, padding, filter_size, input_shape):
		self.name = 'Initial Convolutionl Layer'
		self.filters = filters
		self.padding = padding
		self.filter_size = filter_size
		self.input_shape = input_shape

	def build_layer(self, model):
		model.add(keras.layers.Conv2D(filters=self.filters,
									   kernel_size=self.filter_size,
									   padding=self.padding,
									   activation='relu',
									   kernel_initializer='he_uniform',
										 input_shape=self.input_shape))

	def mutate_parameters(self):
		mutation = randint(0, 2)
		print("Mutating", self.name, "layer:")
		if mutation == 0 and self.filters >= 64:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 1 and self.filters >= 256:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 2 and self.filters <= 128:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		elif mutation == 3 and self.filters <= 256:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)


class Convolutional:
	# __slots__ = ('name', 'filters', 'padding', 'filter_size')

	def __init__(self, filters, padding, filter_size):
		self.name = 'Conv2D'
		self.filters = filters
		self.padding = padding
		self.filter_size = filter_size

	def build_layer(self, model):
		model.add(keras.layers.Conv2D(filters=self.filters,
									   kernel_size=self.filter_size,
									   padding=self.padding,
									   activation='relu',
									   kernel_initializer='he_uniform'))

	def mutate_parameters(self):
		mutation = randint(0, 2)
		print("Mutating", self.name, "layer:")
		if mutation == 0 and self.filters >= 64:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 1 and self.filters >= 256:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters = int(self.filters / 2)
			print("to ", self.filters)
		elif mutation == 2 and self.filters <= 128:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		elif mutation == 3 and self.filters <= 256:
			print("-->changed self.filters from ", self.filters, " ", end="")
			self.filters *= 2
			print("to ", self.filters)
		



class Pooling:
	__slots__ = ('name', 'pool_size')

	def __init__(self, pool_size):
		self.name = 'MaxPooling2D'
		self.pool_size = pool_size
		#self.padding = padding

	def build_layer(self, model):
		if self.name == 'MaxPooling2D':
			model.add(keras.layers.MaxPooling2D(self.pool_size))
		elif self.name == 'AveragePooling2D':
			model.add(keras.layers.AveragePooling2D(self.pool_size))


	def mutate_parameters(self):
		print("No mutation for Pooling layer")
	


class FullyConnected:
	__slots__ = ('name', 'units', 'num_classes')

	def __init__(self, units, num_classes):
		self.name = "FullyConnected"
		self.units = units
		self.num_classes = num_classes

	def build_layer(self, model):
		model.add(keras.layers.Flatten())
		model.add(keras.layers.Dense(units=self.units, activation='relu', kernel_initializer='he_uniform'))
		model.add(keras.layers.Dropout(0.5))
		model.add(keras.layers.Dense(self.num_classes, activation='softmax'))

	def mutate_parameters(self):
		print("Mutating", self.name, "layer:")
		mutation = randint(0, 2)
		if mutation == 0:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units *= 2
			print("to ", self.units)
		elif mutation == 1:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units *= 2
			print("to ", self.units)
		elif mutation == 2:
			print("-->changed self.units from ", self.units, " ", end="")
			self.units /= 2
			print("to ", self.units)


'''
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(self.num_classes, activation='softmax'))
'''


class Dropout:
	__slots__ = ('name', 'rate')

	def __init__(self, rate):
		self.name = "Dropout"
		self.rate = rate

	def build_layer(self, model):
		model.add(keras.layers.Dropout(self.rate))

	def mutate_parameters(self):
		print("Mutating", self.name, "layer:")
		mutation = randint(0, 3)
		if mutation == 0 and self.rate <= 0.85:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate + 0.10
			print("to ", self.rate)
		elif mutation == 1 and self.rate <= 0.90:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate + 0.05
			print("to ", self.rate)
		elif mutation == 2 and self.rate >= 0.15:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate - 0.10
			print("to ", self.rate)
		elif mutation == 3 and self.rate >= 0.10:
			print("-->changed self.rate from ", self.rate, " ", end="")
			self.rate = self.rate - 0.05
			print("to ", self.rate)


## Network

In [11]:
# Network

import tensorflow as tf
import os

from keras.callbacks import Callback

from keras.models import Sequential
from random import randint, choice
from copy import deepcopy
from keras.optimizers import SGD


class Network:
    __slots__ = ('name', 'block_list', 'fitness', 'model')

    def __init__(self, it):
        self.name = 'parent_' + str(it) if it == 0 else 'net_' + str(it)
        self.block_list = []
        self.fitness = None
        self.model = None

    def build_model(self):
        model = Sequential()                                # create model
        for block in self.block_list:
            for layer in block.get_layers():                # build model
                try:
                    layer.build_layer(model)
                except:
                    print("\nINDIVIDUAL ABORTED, CREATING A NEW ONE\n", layer.name, block.index)
                    return -1
        return model

    def train_and_evaluate(self, model, dataset):
        print("Training", self.name)
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(dataset['x_train'],
                            dataset['y_train'],
                            batch_size=dataset['batch_size'],
                            epochs=dataset['epochs'],
                            validation_data=(dataset['x_val'], dataset['y_val']))

        self.model = model                                    # model
        _, acc = model.evaluate(dataset['x_test'], dataset['y_test'], verbose=1)                                 
        self.fitness = acc         
                                   # fitness
        print("SUMMARY OF", self.name)
        print(model.summary())
        print("FITNESS: ", self.fitness)

        model.save(self.name + '.h5')                       # save model
        save_network(self)                                  # save topology, model and fitness

    def asexual_reproduction(self, it, dataset):

        # if the individual already exists, just load it
        if os.path.isfile('net_' + str(it) + '.h5'):
            print("\n-------------------------------------")
            print("Loading individual net_" + str(it))
            print("--------------------------------------\n")
            individual = load_network('net_' + str(it))
            model = tf.keras.models.load_model(individual.name + '.h5')
            print("SUMMARY OF", individual.name)
            print(model.summary())
            print("FITNESS: ", individual.fitness)
            return individual

        # otherwise, create the individual by mutating the parent
        individual = Network(it)

        print("\n-------------------------------------")
        print("\nCreating individual", individual.name)
        print("--------------------------------------\n")

        individual.block_list = deepcopy(self.block_list)           # copy the layer list from parent

        print("----->Strong Mutation")
        individual.block_mutation(dataset)                          # mutate a block
        individual.layer_mutation(dataset)                          # mutate a layer
        individual.parameters_mutation()                            # mutate some parameters

        model = individual.build_model()

        if model == -1:
            return self.asexual_reproduction(it, dataset)

        individual.train_and_evaluate(model, dataset)

        return individual

    def block_mutation(self, dataset):
        print("Block Mutation")

        print([(block.index, block.type) for block in self.block_list])

        # block list containing all the blocks with type = 1
        bl = [block.index for block in self.block_list if block.type == 1]

        if len(bl) == 0:
            print("Creating a new block with two Convolutional layers and a Pooling layer")
            self.block_list[1].index = 2
            layerList1 = [
                Convolutional(filters=pow(2, randint(5, 8)),
                              filter_size=(3, 3),
                              padding='same'),
                Convolutional(filters=pow(2, randint(5, 8)),
                              filter_size=(3, 3),
                              padding='same')
            ]
            layerList2 = [
                Pooling(pool_size=(2, 2))
            ]
            b = Block(1, 1, layerList1, layerList2)
            self.block_list.insert(1, b)
            return

        block_idx = randint(1, max(bl))         # pick a random block among all the blocks with type = 1
        block_type_idx = randint(0, 1)          # 1 -> Conv2D; 0 -> Pooling or Dropout
        mutation_type = randint(0, 1)           # 1 -> remove; 0 -> add

        # list of layers of the selected block
        layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2
        length = len(layerList)

        if mutation_type:                                       # remove
            if length == 1:
                del self.block_list[block_idx]
            elif block_type_idx:
                pos = randint(0, length - 1)
                print("Removing a Conv2D layer at", pos)
                del layerList[pos]
            else:
                pos = randint(0, length - 1)
                print("Removing a Pooling/Dropout layer at", pos)
                del layerList[pos]
        else:                                                   # add
            if block_type_idx:
                print("Inserting a Convolutional layer")
                layer = Convolutional(filters=pow(2, randint(5, 8)),
                                      filter_size=(3, 3),
                                      padding='same')
                layerList.insert(randint(0, length - 1), layer)
            else:
                if randint(0, 1):                               # 1 -> Pooling; 0 -> Dropout
                    print("Inserting a Pooling layer")
                    layer = Pooling(pool_size=(2, 2))
                    layerList.insert(randint(0, length - 1), layer)
                else:
                    print("Inserting a Dropout layer")
                    rate = choice([0.15, 0.25, 0.35, 0.50])
                    layer = Dropout(rate=rate)
                    layerList.insert(randint(0, length - 1), layer)

    def layer_mutation(self, dataset):
        print("Layer Mutation")

        # pick a random block among all the blocks with type = 1
        bl = [block.index for block in self.block_list if block.type == 1]

        if len(bl) == 0:
            return

        block_idx = randint(1, max(bl))
        block_type_idx = randint(0, 1)      # 1 -> Conv2D; 0 -> Pooling or Dropout

        # list of layers of the selected block
        layerList = self.block_list[block_idx].layerList1 if block_type_idx else self.block_list[block_idx].layerList2

        if len(layerList) == 0:
            if block_type_idx:
                layer = Convolutional(filters=pow(2, randint(5, 8)),
                                      filter_size=(3, 3),
                                      padding='same',
                                      input_shape=dataset['x_train'].shape[1:])
                self.block_list[block_idx].layerList1.append(layer)
                return
            else:
                layer = Pooling(pool_size=(2, 2))
                self.block_list[block_idx].layerList2.append(layer)

        idx = randint(0, len(layerList) - 1)
        layer = layerList[idx]

        if layer.name == 'Conv2D':
            print("Splitting Conv2D layer at index", idx)
            layer.filters = int(layer.filters * 0.5)
            layerList.insert(idx, deepcopy(layer))
        elif layer.name == 'MaxPooling2D' or layer.name == 'AveragePooling2D':
            print("Changing Pooling layer at index", idx, "with Conv2D layer")
            del layerList[idx]
            conv_layer = Convolutional(filters=pow(2, randint(5, 8)),
                                       filter_size=(3, 3), padding = 'same')
            layerList.insert(idx, conv_layer)

    def parameters_mutation(self):
        print("Parameters Mutation")
        for block in self.block_list:
            for layer in block.get_layers():
                if randint(0, 1):
                    layer.mutate_parameters()


In [12]:
# inout

import tensorflow as tf
import os
#from utilities import load_network
#from topology import Block, Convolutional, Pooling, Dropout, FullyConnected
#from network import Network


def compute_parent(dataset):
    if os.path.isfile('parent_0.h5'):
        daddy = load_network('parent_0')
        model = tf.keras.models.load_model('parent_0.h5')
        print("Loading parent_0")
        print("SUMMARY OF", daddy.name)
        print(model.summary())
        print("FITNESS:", daddy.fitness)
        return daddy

    daddy = Network(0)

    
    layerList1 = [
        InitConvolutional(filters=32, filter_size=(3, 3),  padding='same', 
                          input_shape=dataset['x_train'].shape[1:]),
        Convolutional(filters=32, filter_size=(3, 3), padding='same')
    ]
    layerList2 = [
        Pooling(pool_size=(2, 2)), 
        Dropout(rate=0.2)
    ]

    daddy.block_list.append(Block(0, 0, layerList1, layerList2))
    
    layerList1 = [
        Convolutional(filters=64, filter_size=(3, 3), padding='same'),
        Convolutional(filters=64, filter_size=(3, 3), padding='same')
    ]
    layerList2 = [
        Pooling(pool_size=(2, 2)),
        Dropout(rate=0.3)
    ]

    daddy.block_list.append(Block(1, 1, layerList1, layerList2))
    
    layerList1 = [
        Convolutional(filters=128, filter_size=(3, 3), padding='same'),
        Convolutional(filters=128, filter_size=(3, 3), padding='same')
    ]
    layerList2 = [
        Pooling(pool_size=(2, 2)),
        Dropout(rate=0.4)
    ]
    daddy.block_list.append(Block(1, 2, layerList1, layerList2))
    
    layerList1 = [
        FullyConnected(units=128, num_classes=dataset['num_classes'])
    ]
    layerList2 = []

    daddy.block_list.append(Block(2, 3, layerList1,layerList2 = []))
    
    '''
    layerList1 = [
        InitConvolutional(filters=32, filter_size=(3, 3), padding='same',
                      input_shape=dataset['x_train'].shape[1:]),
        Convolutional(filters=32, filter_size=(3, 3),  padding='valid')
        ]
    layerList2 = [
        Pooling(pool_size=(2, 2))
    ]
    daddy.block_list.append(Block(0, 0, layerList1, layerList2))

    layerList1 = [
        FullyConnected(units=128, num_classes=dataset['num_classes'])
    ]

    layerList2 = []

    daddy.block_list.append(Block(2, 1, layerList1, layerList2))
    
'''
    model = daddy.build_model()
    daddy.train_and_evaluate(model, dataset)
    return daddy


## Main

In [13]:
# Main

import tensorflow as tf
import os

from random import randint, sample

from copy import deepcopy

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)      # suppress messages from Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def initialize_population(population_size, dataset):
    print("----->Initializing Population")
    daddy = compute_parent(dataset)                                 # load parent from input
    population = [daddy]
    for it in range(1, population_size):
        population.append(daddy.asexual_reproduction(it, dataset))

    # sort population on ascending order based on fitness
    return sorted(population, key=lambda cnn: -cnn.fitness)


def selection(k, population, num_population):
    if k == 0:                                              # elitism selection
        print("----->Elitism selection")
        return population[0], population[1]
    elif k == 1:                                            # tournament selection
        print("----->Tournament selection")
        i = randint(0, num_population - 1)
        j = i
        while j < num_population - 1:
            j += 1
            if randint(1, 100) <= 50:
                return population[i], population[j]
        return population[i], population[0]
    else:                                                   # proportionate selection
        print("----->Proportionate selection")
        cum_sum = 0
        for i in range(num_population):
            cum_sum += population[i].fitness
        perc_range = []
        for i in range(num_population):
            count = int(100 * population[i].fitness / cum_sum)
            for j in range(count):
                perc_range.append(i)
        i, j = sample(range(1, len(perc_range)), 2)
        while i == j:
            i, j = sample(range(1, len(perc_range)), 2)
        return population[perc_range[i]], population[perc_range[j]]


def crossover(parent1, parent2, it):
    print("----->Crossover")
    child = Network(it)

    first, second = None, None
    if randint(0, 1):
        first = parent1
        second = parent2
    else:
        first = parent2
        second = parent1

    child.block_list = deepcopy(first.block_list[:randint(1, len(first.block_list) - 1)]) \
                       + deepcopy(second.block_list[randint(1, len(second.block_list) - 1):])

    order_indexes(child)                            # order the indexes of the blocks

    return child


def genetic_algorithm(num_population, num_generation, num_offspring, dataset):
    print("Genetic Algorithm")

    population = initialize_population(num_population, dataset)

    print("\n-------------------------------------")
    print("Initial Population:")
    for cnn in population:
        print(cnn.name, ': ', cnn.fitness)
    print("--------------------------------------\n")

    # for printing statistics about fitness and number of parameters of the best individual
    stats = [(population[0].fitness, population[0].model.count_params())]

    for gen in range(1, num_generation + 1):

        '''
            k is the selection parameter:
                k = 0 -> elitism selection
                k = 1 -> tournament selection
                k = 2 -> proportionate selection
        '''
        k = randint(0, 2)

        print("\n------------------------------------")
        print("Generation", gen)
        print("-------------------------------------")

        for c in range(num_offspring):

            print("\nCreating Child", c)

            parent1, parent2 = selection(k, population, num_population)                 # selection
            print("Selected", parent1.name, "and", parent2.name, "for reproduction")

            child = crossover(parent1, parent2, c + num_population)                     # crossover
            print("Child has been created")

            print("----->Soft Mutation")
            child.layer_mutation(dataset)                                               # mutation
            child.parameters_mutation()
            print("Child has been mutated")

            model = child.build_model()                                                 # evaluation

            while model == -1:
                child = crossover(parent1, parent2, c + num_population)
                child.block_mutation(dataset)
                child.layer_mutation(dataset)
                child.parameters_mutation()
                model = child.build_model()

            child.train_and_evaluate(model, dataset)

            if child.fitness > population[-1].fitness:                                  # evolve population
                print("----->Evolution: Child", child.name, "with fitness", child.fitness, "replaces parent ", end="")
                print(population[-1].name, "with fitness", population[-1].fitness)
                name = population[-1]
                population[-1] = deepcopy(child)
                population[-1].name = name
                population = sorted(population, key=lambda net: -net.fitness)
            else:
                print("----->Evolution: Child", child.name, "with fitness", child.fitness, "is discarded")

        stats.append((population[0].fitness, population[0].model.count_params()))

    print("\n\n-------------------------------------")
    print("Final Population")
    print("-------------------------------------\n")
    for cnn in population:
        print(cnn.name, ': ', cnn.fitness)

    print("\n-------------------------------------")
    print("Stats")
    for i in range(len(stats)):
        print("Best individual at generation", i + 1, "has fitness", stats[i][0], "and parameters", stats[i][1])
    print("-------------------------------------\n")

    # plot the fitness and the number of parameters of the best individual at each iteration
    plot_statistics(stats)

    return population[0]


def main():
    batch_size = 24                         # the number of training examples in one forward/backward pass
    num_classes = 2                         # number of dataset classes
    epochs = 3                              # number of forward and backward passes of all the training examples

    '''
        dataset contains the hyper parameters for loading data and the dataset:
            dataset = {
                'batch_size': batch_size,
                'num_classes': num_classes,
                'epochs': epochs,
                'x_train': x_train,
                'x_test': x_test,
                'y_train': y_train,
                'y_test': y_test,
                'x_val': x_val,
                'y_val': y_val
            }
    '''
    dataset = load_dataset(X, y, batch_size, num_classes, epochs)

    num_population = 4
    num_generation = 2
    num_offspring = 2

    # plot the best model obtained
    optCNN = genetic_algorithm(num_population, num_generation, num_offspring, dataset)

    # plot the training and validation loss and accuracy
    num_epoch = 3
    model = optCNN.build_model()
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(dataset['x_train'],
                        dataset['y_train'],
                        batch_size=dataset['batch_size'],
                        epochs=num_epoch,
                        validation_data=(dataset['x_val'], dataset['y_val']),
                        verbose=2)
    

    optCNN.model = model                                         # model
    _, acc = model.evaluate(dataset['x_test'], dataset['y_test'], verbose=1)  
    
    optCNN.fitness = acc                                          # fitness
    
   

    print("\n\n-------------------------------------")
    print("The initial CNN has been evolved successfully in the individual", optCNN.name)
    print("-------------------------------------\n")
    daddy = load_network('parent_0')
    model = tf.keras.models.load_model('parent_0.h5')
    print("\n\n-------------------------------------")
    print("Summary of initial CNN")
    print(model.summary())
    print("Fitness of initial CNN:", daddy.fitness)

    print("\n\n-------------------------------------")
    print("Summary of evolved individual")
    print(optCNN.model.summary())
    print("Fitness of the evolved individual:", optCNN.fitness)
    print("-------------------------------------\n")

    plot_training(history)



In [ ]:
main()

Genetic Algorithm
----->Initializing Population
Loading parent_0
SUMMARY OF parent_0
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 200, 200, 32)      320       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 100, 100, 64)      36928     
___________________________________